In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import librosa

# --- 1. Load the Trained Model ---
# Make sure this path is correct
model_path = '/content/drive/MyDrive/Emotion Recognition from Speech/emotion_recognition_model.h5'
loaded_model = load_model(model_path)


In [6]:

# --- 2. Re-create the LabelEncoder ---
# You need to define the class names in the correct order as they were during training.
# This order is based on the RAVDESS dataset emotion_mapping.
class_names = ['angry', 'calm', 'disgust', 'fearful', 'happy', 'neutral', 'sad', 'surprised']
label_encoder = LabelEncoder()
label_encoder.fit(class_names)

# --- 3. Preprocess Your Sample Data ---
# Re-use the feature extraction function
def extract_features(audio_path, mfcc=True, chroma=True, mel=True):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        features = []
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
            features.append(mfccs)
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
            features.append(chroma)
        if mel:
            mel_spec = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
            features.append(mel_spec)
        return np.hstack(features)
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None



In [12]:
# Path to your new audio file. Update this path.
sample_audio_path = '/content/drive/MyDrive/Emotion Recognition from Speech/Sample_data/-128-ytshorts.savetube.me.wav'
features = extract_features(sample_audio_path)

# Reshape for the model
if features is not None:
    features_reshaped = np.expand_dims(features, axis=0)
    features_reshaped = np.expand_dims(features_reshaped, axis=2)

    # --- 4. Make a Prediction and Interpret the Results ---
    prediction = loaded_model.predict(features_reshaped)

    predicted_class_index = np.argmax(prediction)
    predicted_emotion = label_encoder.classes_[predicted_class_index]
    confidence = np.max(prediction) * 100

    print(f"The predicted emotion is: {predicted_emotion}")
    print(f"Confidence: {confidence:.2f}%")
else:
    print("Feature extraction failed, cannot make a prediction.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
The predicted emotion is: fearful
Confidence: 100.00%
